We define the Needle-in-Sequence environment as a finite episodic MDP where the state consists of the observed sequence and hidden needle index. The reward function is deterministic. Although the transition function is easy to compute in this environment, we explicitly represent it to enable future model-based RL. The agent maintains estimates of
𝑅(𝑠,𝑎) and 𝑃(𝑠′∣𝑠,𝑎) to allow dynamic programming.

           ┌───────────────────────────┐
           │     Environment           │
           │  X, hidden index          │
           └─────────────┬─────────────┘
                         │
                         ▼
                 Observation: X
                         │
                         ▼
               ┌────────────────┐
               │      Agent     │
               │ Hierarchical   │
               │   Attention    │
               └───────┬────────┘
                       │
                 Action: pick index a
                       │
                       ▼
           ┌───────────────────────────┐
           │ Reward: r = 1 if a==i     │
           │         r = 0 otherwise   │
           └───────────────────────────┘


In [3]:
import os
import json
import math
from dataclasses import dataclass, asdict
from typing import Dict, Any, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F


class NeedleInSequenceEnv:
    """
    Observation: X in R^{T x d}
    Hidden needle index i*
    Action: choose index a in {0..T-1}
    Reward: 1 if a==i* else 0
    Episode length: 1 step
    """
    def __init__(self, T=64, d=16, signal_strength=6.0, device="cpu"):
        self.T = T
        self.d = d
        self.signal_strength = signal_strength
        self.device = device
        self.X = None
        self.needle_idx = None
        self.needle_vec = None

    def reset(self):
        self.X = torch.randn(self.T, self.d, device=self.device)
        self.needle_idx = int(torch.randint(0, self.T, (1,), device=self.device).item())
        self.needle_vec = torch.randn(self.d, device=self.device)
        self.X[self.needle_idx] += self.signal_strength * self.needle_vec
        return self.X.clone()

    def step(self, action: int):
        reward = 1.0 if action == self.needle_idx else 0.0
        done = True
        info = {"needle_idx": self.needle_idx}
        return self.X.clone(), reward, done, info


class HierarchicalAttentionEncoder(nn.Module):
    """
    Uses Blocks in a hierarchy, to determine which tokens would be attended using a top-k approach
      - block summaries
      - select top-k blocks
      - attend within chosen blocks (top-k tokens)
    """
    def __init__(self, d: int, block_size: int = 8, k_blocks: int = 2, k_tokens: int = 4):
        super().__init__()
        self.d = d
        self.block_size = block_size
        self.k_blocks = k_blocks
        self.k_tokens = k_tokens

        self.Wq = nn.Linear(d, d)
        self.Wk = nn.Linear(d, d)
        self.Wv = nn.Linear(d, d)

    def forward(self, X: torch.Tensor) -> torch.Tensor:

        T, d = X.shape
        assert d == self.d
        assert T % self.block_size == 0,

        Q = self.Wq(X)
        K = self.Wk(X)
        V = self.Wv(X)

        num_blocks = T // self.block_size


        K_blocks = K.view(num_blocks, self.block_size, d)
        V_blocks = V.view(num_blocks, self.block_size, d)


        block_keys = K_blocks.mean(dim=1)

        contexts = []

        for i in range(T):
            q = Q[i]

            # Choose top-k blocks
            block_scores = (q @ block_keys.T) / math.sqrt(d)
            chosen_blocks = torch.topk(block_scores, self.k_blocks).indices

            c_i = torch.zeros(d, device=X.device)

            # Local attention
            for b in chosen_blocks:
                Kb = K_blocks[b]
                Vb = V_blocks[b]

                token_scores = (q @ Kb.T) / math.sqrt(d)
                chosen_tokens = torch.topk(token_scores, self.k_tokens).indices

                w = F.softmax(token_scores[chosen_tokens], dim=0)
                c_i += (w.unsqueeze(-1) * Vb[chosen_tokens]).sum(dim=0)

            contexts.append(c_i)

        c = torch.stack(contexts, dim=0).mean(dim=0)  # (d,)
        return c


class HierAttnPolicyValueNet(nn.Module):

    def __init__(self, d: int, T: int, block_size=8, k_blocks=2, k_tokens=4, hidden=128):
        super().__init__()
        self.T = T
        self.encoder = HierarchicalAttentionEncoder(d, block_size, k_blocks, k_tokens)

        self.policy_head = nn.Sequential(
            nn.Linear(d, hidden),
            nn.ReLU(),
            nn.Linear(hidden, T)
        )
        self.value_head = nn.Sequential(
            nn.Linear(d, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, X: torch.Tensor):

        c = self.encoder(X)                # (d,)
        logits = self.policy_head(c)       # (T,)
        value = self.value_head(c).squeeze(-1)  # scalar
        return logits, value



@dataclass
class AgentArgs:
    T: int = 64
    d: int = 16
    block_size: int = 8
    k_blocks: int = 2
    k_tokens: int = 4

    signal_strength: float = 6.0
    lr: float = 1e-3
    episodes: int = 5000
    eval_episodes: int = 1000
    seed: int = 0
    device: str = "cpu"

    artifacts_dir: str = "artifacts"
    run_name: str = "needle_hierattn_rl"


class HierAttnRLAgent:
    """
    Trains a hierarchical-attention policy with reinforcement baseline
    Saves best policy and weights.
    """
    def __init__(self, args: AgentArgs):
        self.args = args
        torch.manual_seed(args.seed)

        self.env = NeedleInSequenceEnv(
            T=args.T, d=args.d, signal_strength=args.signal_strength, device=args.device
        )

        self.model = HierAttnPolicyValueNet(
            d=args.d, T=args.T,
            block_size=args.block_size, k_blocks=args.k_blocks, k_tokens=args.k_tokens
        ).to(args.device)

        self.opt = torch.optim.Adam(self.model.parameters(), lr=args.lr)

        os.makedirs(args.artifacts_dir, exist_ok=True)
        self.ckpt_prefix = os.path.join(args.artifacts_dir, args.run_name)

        self.best_score = -1.0

    @torch.no_grad()
    def act(self, X: torch.Tensor, greedy: bool = False) -> int:
        logits, _ = self.model(X)
        if greedy:
            return int(torch.argmax(logits).item())
        probs = F.softmax(logits, dim=-1)
        a = int(torch.distributions.Categorical(probs).sample().item())
        return a

    @torch.no_grad()
    def evaluate(self, episodes: int, greedy: bool = True) -> float:
        correct = 0.0
        for _ in range(episodes):
            X = self.env.reset()
            a = self.act(X, greedy=greedy)
            _, r, _, _ = self.env.step(a)
            correct += r
        return correct / episodes

    def save_best(self) -> None:
        torch.save(self.model.state_dict(), self.ckpt_prefix + "_best.pt")
        meta = {"best_score": float(self.best_score), "args": asdict(self.args)}
        with open(self.ckpt_prefix + "_meta.json", "w", encoding="utf-8") as f:
            json.dump(meta, f, indent=2)

    def load_best(self) -> bool:
        path = self.ckpt_prefix + "_best.pt"
        if not os.path.exists(path):
            return False
        self.model.load_state_dict(torch.load(path, map_location=self.args.device))
        meta_path = self.ckpt_prefix + "_meta.json"
        if os.path.exists(meta_path):
            with open(meta_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
            self.best_score = float(meta.get("best_score", self.best_score))
        return True

    def train(self) -> Dict[str, Any]:

        for ep in range(1, self.args.episodes + 1):
            X = self.env.reset()

            logits, value = self.model(X)
            probs = F.softmax(logits, dim=-1)
            dist = torch.distributions.Categorical(probs)
            a = dist.sample()

            _, r, _, _ = self.env.step(int(a.item()))
            reward = torch.tensor(r, device=self.args.device, dtype=torch.float32)

            advantage = (reward - value.detach())

            policy_loss = -dist.log_prob(a) * advantage
            value_loss = F.mse_loss(value, reward)
            loss = policy_loss + 0.5 * value_loss

            self.opt.zero_grad()
            loss.backward()
            self.opt.step()

            if ep % 200 == 0:
                score = self.evaluate(self.args.eval_episodes, greedy=True)
                if score > self.best_score:
                    self.best_score = score
                    self.save_best()

                print(f"ep={ep:5d}  loss={loss.item():.4f}  eval_acc={score:.3f}  best={self.best_score:.3f}")

        final_score = self.evaluate(self.args.eval_episodes, greedy=True)
        return {"final_eval_acc": final_score, "best_eval_acc": self.best_score}


def run(mode: str, args: AgentArgs):
    agent = HierAttnRLAgent(args)

    if mode == "train":
        out = agent.train()
        print("[TRAIN DONE]", out)

    elif mode == "eval":
        if not agent.load_best():
            raise RuntimeError("No saved best model found. Run train first.")
        score = agent.evaluate(args.eval_episodes, greedy=True)
        print("[EVAL] acc:", score, "best_score:", agent.best_score)

    else:
        raise ValueError("mode must be 'train' or 'eval'")


if __name__ == "__main__":
    args = AgentArgs(
        T=64, d=16,
        block_size=8, k_blocks=2, k_tokens=4,
        signal_strength=6.0,
        lr=1e-3,
        episodes=3000,
        eval_episodes=500,
        device="cpu",
        run_name="needle_hierattn_rl_v1"
    )

    run("train", args)
    run("eval", args)


ep=  200  loss=-0.8545  eval_acc=0.016  best=0.016
ep=  400  loss=-0.5232  eval_acc=0.014  best=0.016
ep=  600  loss=-0.4469  eval_acc=0.010  best=0.016
ep=  800  loss=0.2501  eval_acc=0.012  best=0.016
ep= 1000  loss=0.1940  eval_acc=0.010  best=0.016
ep= 1200  loss=0.0709  eval_acc=0.022  best=0.022
ep= 1400  loss=-0.0241  eval_acc=0.018  best=0.022
ep= 1600  loss=0.1074  eval_acc=0.006  best=0.022
ep= 1800  loss=-0.0161  eval_acc=0.010  best=0.022
ep= 2000  loss=-0.2611  eval_acc=0.012  best=0.022
ep= 2200  loss=-0.0409  eval_acc=0.016  best=0.022
ep= 2400  loss=-0.2793  eval_acc=0.020  best=0.022
ep= 2600  loss=-0.0238  eval_acc=0.026  best=0.026
ep= 2800  loss=-0.0817  eval_acc=0.014  best=0.026
ep= 3000  loss=-0.1408  eval_acc=0.020  best=0.026
[TRAIN DONE] {'final_eval_acc': 0.006, 'best_eval_acc': 0.026}
[EVAL] acc: 0.012 best_score: 0.026
